## Preprocessing

In [ ]:
# Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns= ['EIN', 'NAME'], axis=1)
application_df.head(2)

In [ ]:
# Number of unique values in each column.
unique_values = application_df.nunique()
print(unique_values)

In [ ]:
# APPLICATION_TYPE value counts for binning
app_type_valuecount = application_df['APPLICATION_TYPE'].value_counts()
print(app_type_valuecount)

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
apptype_cutoff_value = 500
application_types_to_replace = app_type_valuecount[app_type_valuecount < apptype_cutoff_value].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_valuecounts = application_df['CLASSIFICATION'].value_counts()
print(classification_valuecounts)

In [ ]:
# CLASSIFICATION value counts >1
classification_valuecounts[classification_valuecounts > 1]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
cutoff = 1800
classifications_to_replace = classification_valuecounts[classification_valuecounts < cutoff].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Dummies for categorical data
converted_df = pd.get_dummies(application_df)
converted_df.head(2)

In [ ]:
# Split preprocessed data into features and target arrays
y = converted_df['IS_SUCCESSFUL'].values
X = converted_df.drop(['IS_SUCCESSFUL'], axis=1).values

# Split training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

In [ ]:
# Instance of new model
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim = input_features))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_model.summary()

In [ ]:
# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn_model.save('Attempt1_AlphabetSoupCharity.h5')